In [2]:
import pandas as pd
dataset = pd.read_csv('wine-reviews\winemag-data-130k-v2.csv')
# dataset.head()

In [7]:
#Updating the Dataset

stats = dataset['price'].describe()
upper_quartile = stats[6]
lower_quartile = stats[4]
iqr = upper_quartile - lower_quartile
dataset_updated = dataset[dataset.price<=upper_quartile+1.5*iqr]

X_t12 = dataset_updated.drop(["Unnamed: 0", "taster_twitter_handle","points"], axis = 1)
y_t12 = dataset_updated["points"]

In [12]:
from sklearn.model_selection import train_test_split
import numpy as np

# Data Imputation
X_train, X_test, y_train, y_test = train_test_split(X_t12, y_t12)
#imputation
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], index=X.columns)
        return self

    def transform(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], index=X.columns)
        return X.fillna(self.fill)

X_train = DataFrameImputer().fit_transform(X_train)
X_test = DataFrameImputer().transform(X_test)


In [13]:
#Creating new feature combining text data features
X_train["description+title"] = X_train["description"] + X_train["title"]

In [16]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

model_1 = Word2Vec(X_train["description+title"], size=300, min_count=1)

In [27]:
words = list(model_1.wv.vocab)
print(words)

['E', 's', 't', 'a', 'e', '-', 'g', 'r', 'o', 'w', 'n', ' ', 'R', 'u', '(', '6', '3', '%', ')', ',', 'V', 'i', '2', 'd', 'C', 'h', 'y', '5', 'b', 'l', 'm', 'c', 'f', 'p', 'v', '.', 'F', 'W', '0', '1', 'A', 'G', 'S', 'é', "'", 'j', 'T', 'D', 'k', 'L', '9', 'M', 'P', '*', 'ô', '/', 'H', 'B', '8', 'z', 'I', 'N', 'ñ', '4', 'O', 'x', 'è', 'Y', 'Z', ';', 'q', 'K', 'Ü', 'ü', 'U', 'J', '“', '”', 'í', '7', '–', 'â', ':', 'É', 'ç', 'ã', 'ú', 'ö', '&', 'X', '’', 'û', 'Q', 'á', 'ó', '—', 'ê', '°', 'ÿ', 'à', 'ä', '$', '!', 'À', 'ì', 'Ú', '•', 'ò', 'ć', 'ù', '?', 'î', 'Î', '\r', '\n', '+', 'Ö', 'ë', '=', 'ø', 'å', '\xa0', '#', 'ű', 'ő', '…', '\xad', 'ß', 'ï', 'Á', 'ș', '"', 'Õ', 'ý', 'ǎ', '‘', 'Ç', 'Ì', 'č', 'š', 'Š', 'Ò', '¬', 'ğ', '\x9d', '[', ']', 'ş', 'È', 'œ', 'Č', 'ž', 'Ž', 'ō', 'õ', 'Ä', '´', 'Ã', '¨', 'Ó', '\u3000', 'Â', 'º', 'ı', '¡', 'ā', 'Ñ', '▪', '@', '|', 'ă', '`']


In [28]:
model_1.wv.vocab

{'E': <gensim.models.keyedvectors.Vocab at 0x22f28f78308>,
 's': <gensim.models.keyedvectors.Vocab at 0x22f315d6488>,
 't': <gensim.models.keyedvectors.Vocab at 0x22f315d65c8>,
 'a': <gensim.models.keyedvectors.Vocab at 0x22f315d6408>,
 'e': <gensim.models.keyedvectors.Vocab at 0x22f315d6588>,
 '-': <gensim.models.keyedvectors.Vocab at 0x22f315d6648>,
 'g': <gensim.models.keyedvectors.Vocab at 0x22f315d6688>,
 'r': <gensim.models.keyedvectors.Vocab at 0x22f315d66c8>,
 'o': <gensim.models.keyedvectors.Vocab at 0x22f315d6548>,
 'w': <gensim.models.keyedvectors.Vocab at 0x22f315d6608>,
 'n': <gensim.models.keyedvectors.Vocab at 0x22f315d6708>,
 ' ': <gensim.models.keyedvectors.Vocab at 0x22f315d6748>,
 'R': <gensim.models.keyedvectors.Vocab at 0x22f315d6788>,
 'u': <gensim.models.keyedvectors.Vocab at 0x22f315d67c8>,
 '(': <gensim.models.keyedvectors.Vocab at 0x22f315d6808>,
 '6': <gensim.models.keyedvectors.Vocab at 0x22f315d6848>,
 '3': <gensim.models.keyedvectors.Vocab at 0x22f315d6888

In [66]:
X = X_train["description+title"].tolist()
for i in range(len(X)):
    X[i] = X[i].split()

In [67]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

model = Word2Vec(X, size=300, min_count=1)

In [78]:
words = list(model.wv.vocab)
print(words)

['Estate-grown', 'Roussanne', '(63%),', 'Viognier', '(32%)', 'and', 'Chardonnay', '(5%)', 'are', 'blended', 'together', 'in', 'a', 'medium-bodied', 'coming-together', 'of', 'floral,', 'light', 'aromatics', 'supportive', 'acidity.', 'Fruity', 'highlights', 'run', 'the', 'gamut', 'apricot,', 'peach', 'lime,', 'with', 'tiny', 'touch', 'sweetness', 'on', 'finish.Westwood', '2015', 'Annadel', 'Gap', 'Vineyard', 'White', '(Sonoma', 'Valley)', 'A', 'very', 'pale', 'rosé,', 'this', "wine's", 'delicate', 'aromas', 'flowers,', 'crushed', 'stone,', 'canteloupe', 'conjure', 'images', 'summer.', 'The', 'flavors', 'follow', 'similar', 'vein,', 'carried', 'by', 'plenty', 'weight', 'long,', 'slightly', 'soft', 'finish.', 'Drink', 'now.', '1,000', 'cases', 'imported.Domaine', 'Sainte', 'Lucie', '2009', 'MiP*', 'Made', 'Provence', 'Rosé', '(Côtes', 'de', 'Provence)', 'This', 'estate-grown,', 'organic', 'is', 'terrific', 'value,', 'detailed', 'brightly', 'fruity,', 'precise', 'angles', 'spicy', 'details.

In [73]:
print(model[words].shape)

C:\Users\himab\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(141404, 300)


In [79]:
sentences = X_train["description+title"].tolist()
sentences

['Estate-grown Roussanne (63%), Viognier (32%) and Chardonnay (5%) are blended together in a medium-bodied coming-together of floral, light aromatics and supportive acidity. Fruity highlights run the gamut of apricot, peach and lime, with a tiny touch of sweetness on the finish.Westwood 2015 Annadel Gap Vineyard White (Sonoma Valley)',
 "A very pale rosé, this wine's delicate aromas of flowers, crushed stone, canteloupe and peach conjure images of summer. The flavors follow in a similar vein, carried by plenty of weight and a long, slightly soft finish. Drink now. 1,000 cases imported.Domaine Sainte Lucie 2009 MiP* Made in Provence Rosé (Côtes de Provence)",
 'This estate-grown, organic Chardonnay is a terrific value, detailed and brightly fruity, with precise angles and spicy details. The apple/pear/pineapple fruit flavors are crisply defined, perfectly ripe, and beautifully balanced.Thistle 2012 Chardonnay (Dundee Hills)',
 "This interesting Chenin-based blend, also made with 15% Sau

In [ ]:
#average word2vec for each row
sent_vectors = [] #avg w2v for each row is stored in this list
for sent in sentences:
    sent_vec = np.zeros(300)
    cnt_words = 0 #num of words with a valid vector in the sentence
    for word in sent:
        if word in words:
            vec = model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

In [64]:
X_input = model[model.wv.vocab]

C:\Users\himab\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [65]:
X_input.shape

(172, 300)

In [19]:
model_rr = Ridge(alpha = 10)
scores_rr = cross_val_score(model_rr, X, y_train)

np.mean(scores_rr)

array([[ 2.1611598 ,  2.4449117 , -1.7608999 , ...,  0.5037382 ,
        -0.30133265,  0.34917232],
       [ 0.2701056 , -1.1764582 ,  1.0284036 , ..., -0.04696498,
        -0.11751584,  0.05320783],
       [ 0.6513114 , -1.0175353 ,  1.1038789 , ...,  0.17116548,
        -0.02812906,  0.10688964],
       ...,
       [ 0.01681261, -0.00953468,  0.01365416, ...,  0.00566023,
        -0.00355417, -0.00376183],
       [-0.07439891, -0.00538537,  0.02535722, ..., -0.00686798,
         0.04960396, -0.05306564],
       [ 0.01345283, -0.02458967,  0.00933022, ...,  0.02234049,
        -0.05543407,  0.01710752]], dtype=float32)

In [77]:
X_train.shape

(85300, 12)